<a href="https://colab.research.google.com/github/ekity1002/DL4US_FINAL_WORK/blob/master/%E5%89%8D%E5%87%A6%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Mecab のセットアップ

In [0]:
# mecab 
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl lib

In [0]:
# mecab-ipadic-NEologdのインストール
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-NEologd] :     diff => ok
[install-mecab-ipadic-NEologd] :   

In [0]:
#ユーザ辞書用のフォルダ作成
!mkdir ../var/lib/mecab/dic/userdic

In [0]:
import MeCab
import subprocess

def owakati():
    cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
    path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                            shell=True).communicate()[0]).decode('utf-8')
    return MeCab.Tagger("-Owakati -d {0}".format(path))

def mecab_tagger():
    cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
    path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                            shell=True).communicate()[0]).decode('utf-8')
    return MeCab.Tagger("-d {0}".format(path))

m = mecab_tagger()

In [0]:
#追加のユーザ辞書作成。上と同じフォルダに追加
# ! cp "/content/drive/My Drive/Colab Notebooks/data/ingredient_dict.csv" 
# import pandas as pd 
# #df_userdic = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/ingredient_dict.csv",header=0, encoding='utf-8') 
# #df_userdicはIPA品詞体系と同じ並び
# df_userdic.to_csv('../content/userdic.csv',header=None,index=False
# ,encoding='utf-8')

## 画像展開

In [0]:
! mkdir train_img
! unzip "/content/drive/My Drive/Colab Notebooks/data/train_img.zip" -d train_img
! mkdir test_img
! unzip "/content/drive/My Drive/Colab Notebooks/data/test_img.zip" -d test_img

Archive:  /content/drive/My Drive/Colab Notebooks/data/train_img.zip
  inflating: train_img/img1000.jpg   
  inflating: train_img/img1001.jpg   
  inflating: train_img/img1002.jpg   
  inflating: train_img/img1003.jpg   
  inflating: train_img/img1004.jpg   
  inflating: train_img/img1005.jpg   
  inflating: train_img/img1006.jpg   
  inflating: train_img/img1007.jpg   
  inflating: train_img/img1008.jpg   
  inflating: train_img/img1009.jpg   
  inflating: train_img/img100.jpg    
  inflating: train_img/img1010.jpg   
  inflating: train_img/img1011.jpg   
  inflating: train_img/img1012.jpg   
  inflating: train_img/img1013.jpg   
  inflating: train_img/img1014.jpg   
  inflating: train_img/img1015.jpg   
  inflating: train_img/img1016.jpg   
  inflating: train_img/img1017.jpg   
  inflating: train_img/img1018.jpg   
  inflating: train_img/img1019.jpg   
  inflating: train_img/img101.jpg    
  inflating: train_img/img1020.jpg   
  inflating: train_img/img1021.jpg   
  inflating: train_

# 前処理

## 正規化ワード定義

In [0]:
    NORMALIZE_DICT = {
        '粗': 'アラ',
        'アジ': '鯵',
        'あじ': '鯵',
        '真鯵': '鯵',
        'あさり': 'アサリ',
        '浅蜊': 'アサリ',
        '栃尾の油揚げ': '油揚げ',
        '粗挽き黒胡椒': '黒胡椒',
        '粗びき黒胡椒': '黒胡椒',
        'あらびき黒胡椒': '黒胡椒',
        'あらびき黒こしょう': '黒胡椒',
        '粗挽き黒コショウ': '黒胡椒',
        'あらびき胡椒': 'こしょう',
        'アンチョビー': 'アンチョビ',
        'いしる': 'いしり',
        'いり胡麻': '炒りごま',
        'いりごま' : '炒りごま',
        '猪': '猪肉',
        'いんげん豆': 'インゲン',
        'いわし': '鰯',
        'イワシ': '鰯',
        '梅酒の梅': '梅',
        '冷凍枝豆': '枝豆',
        '冷凍海老': 'エビ',
        'えび': 'エビ',
        '赤海老': 'エビ',
        'おからパウダー': 'おから',
        '生おから': 'おから',
        'お刺身': '刺身',
        '炒りゴマ': '炒りごま',
        'いりゴマ': '炒りごま',
        'ブラックペッパー': '黒胡椒',
        '甘夏': 'みかん',
        'トウガラシ': '唐辛子',
        'とうがらし': '唐辛子',
        '万願寺とうがらし': '唐辛子',
        'カツオ': '鰹',
        'かつお': '鰹',
        'かつお節': '鰹節',
        'かつおぶし': '鰹節',
        'カツオ節': '鰹節',
        '花鰹': '鰹節',
        '鰹のタタキ': '鰹のたたき',
        'カツオだし': 'かつおだし',
        '鰹だし': 'かつおだし',
        '鰹出汁': 'かつおだし',
        'かんたん酢': '酢',
        '削り節': '鰹節',
        '弘岡かぶ': 'かぶ',
        '聖護院蕪': 'かぶ',
        '小カブ': 'かぶ',
        '蕪': 'かぶ',
        'きゅうり': 'キュウリ',
        '胡瓜': 'キュウリ',
        'キャベツの葉': 'キャベツ',
        'ケッパー': 'ケイパー',
        '胡麻': 'ゴマ',
        'ごま': 'ゴマ',
        '胡麻油': 'ゴマ油',
        'ごま油': 'ゴマ油',
        'すりゴマ': 'ゴマ',
        '牛蒡': 'ごぼう',
        'ゴボウ': 'ごぼう',
        '板こんにゃく': 'こんにゃく',
        '蒟蒻': 'こんにゃく',
        '糸こんにゃく': 'こんにゃく',
        'さとう': '砂糖',
        'きび砂糖': '砂糖',
        '焼き鮭': '鮭',
        '鮭の切り身': '鮭',
        '銀鮭': '鮭',
        'しし唐': 'ししとう',
        '獅子唐': 'ししとう',
        'アジシオ': '塩',
        '美味しい塩': '塩',
        '塩胡椒': '塩コショウ',
        '塩こしょう': '塩コショウ',
        '塩 こしょう': '塩コショウ',
        '塩 コショウ': '塩コショウ',
        '塩・胡椒': '塩コショウ',
        '塩 胡椒': '塩コショウ',
        '塩・黒胡椒': '塩コショウ',
        '白こしょう': '白胡椒',
        '白コショウ': '白胡椒',
        'ジャガイモ': 'じゃがいも',
        'じゃが芋': 'じゃがいも',
        'シメジ': 'しめじ',
        '胡椒': 'こしょう',
        'シソ': 'しそ',
        '紫蘇': 'しそ',
        'セリ': 'せり',
        'コショウ': 'こしょう',
        '粉こしょう': 'こしょう',
        'コチジャン': 'コチュジャン',
        'ごはん': 'ご飯',
        '温かいご飯': 'ご飯',
        'こんぶ': '昆布',
        'コンブ': '昆布',
        'さば': '鯖',
        'サバ': '鯖',
        'サワラ': '鰆',
        'しょう油': '醤油',
        'しょうゆ': '醤油',
        'しょうが': '生姜',
        '生姜チューブ': 'チューブ生姜',
        '生姜スライス': '生姜',
        '生姜みじん切り': '生姜',
        '生姜すりおろし': 'すりおろし生姜',
        '生姜のすりおろし': 'すりおろし生姜',
        'しょうがのすりおろし': 'すりおろし生姜',
        '七味': '七味唐辛子',
        '七味とうがらし': '七味唐辛子',
        '一味': '一味唐辛子',
        '一味とうがらし': '一味唐辛子',
        '椎茸': 'シイタケ',
        'しいたけ': 'シイタケ',
        'うすくち醤油': '薄口醤油',
        '薄口しょうゆ': '薄口醤油',
        'こいくち醤油': '濃口醤油',
        '濃口しょうゆ': '濃口醤油',
        '鷹の爪輪切り': '鷹の爪',
        'タカノツメ': '鷹の爪',
        'たまねぎ': '玉ねぎ',
        '玉ねぎ みじん切り': '玉ねぎ',
        '玉ねぎみじん切り': '玉ねぎ',
        'タマネギ': '玉ねぎ',
        '玉葱': '玉ねぎ',
        '玉子': '卵',
        'ゆで玉子': 'ゆで卵',
        'ゆで玉子': 'ゆで卵',
        'ゆでたまご': 'ゆで卵',
        '茹でタコ': 'タコ',
        '蒸しタコ': 'タコ',
        'たこ': 'タコ',
        '蛸': 'タコ',
        'だこ': 'タコ',
        'タラ': '鱈',
        '筍': 'たけのこ',
        'ダシ': '出汁',
        '焼きちくわ': '竹輪',
        'ツナ缶詰': 'ツナ',
        'ツナ缶': 'ツナ',
        '鶏がらスープの素': '鶏ガラスープの素',
        '顆粒鶏ガラスープ': '鶏ガラスープの素',
        '鶏がらスープのもと': '鶏ガラスープの素',
        '鶏ムネ肉': '鶏むね肉',
        'ミニトマト': 'トマト',
        'プチトマト': 'トマト',
        'トマト水煮': 'トマト',
        'トマト缶': 'トマト',
        'トマトケチャップ': 'ケチャップ',
        '軟骨': 'なんこつ' ,
        'ナンコツ': 'なんこつ',
        'ニンジン': 'にんじん',
        '人参': 'にんじん',
        '葱': 'ねぎ',
        'ネギ': 'ねぎ',
        '長ネギ': 'ねぎ',
        '九条ネギ': 'ねぎ',
        '九条ねぎ': 'ねぎ',
        '刻みねぎ': 'ねぎ　',
        '刻み葱': 'ねぎ',
        '下仁田ネギ': 'ねぎ',
        '白髪ねぎ': 'ねぎ',
        '細ネギ': 'ねぎ',
        '長ネギのみじん切り': 'ねぎ',
        'なす': 'ナス',
        '茄子': 'ナス',
        '水なす': 'ナス',
        '長茄子': 'ナス',
        '水茄子': 'ナス',
        'にんにく': 'ニンニク',
        '大蒜': 'ニンニク',
        'ニンニクチューブ': 'チューブニンニク',
        'にんにくチューブ': 'チューブニンニク',
        'チューブにんにく': 'チューブニンニク',
        'にんにく(チューブ)': 'チューブニンニク',
        'にんにくのすりおろし': 'ニンニク',
        'にんにくみじん切り': 'ニンニク',
        '韮': 'ニラ',
        'にら': 'ニラ',
        '日本ハム': 'ハム',
        'ハチミツ': 'はちみつ',
        '蜂蜜': 'はちみつ',
        '味醂': 'みりん',
        '破竹': '淡竹',
        '紅生姜': '紅しょうが',
        '豚バラ塊肉': '豚バラ',
        '豚バラ肉ブロック': '豚バラ',
        '豚バラ薄切り肉': '豚バラ',
        '豚バラ肉': '豚バラ',
        'ふぐ': '河豚',
        'フグ': '河豚',
        'ベーコン(ブロック)': 'ベーコン',
        'カリカリベーコン': 'ベーコン',
        'ベビーホタテ': 'ホタテ',
        'ひじきの煮物': 'ひじき',
        '帆立': 'ホタテ',
        'ほたて': 'ホタテ',
        '乾燥パセリ': 'パセリ',
        'イタリアンパセリ': 'パセリ',
        'パセリのみじん切り': 'パセリ',
        'パセリみじん切り': 'パセリ',
        '白菜キムチ': 'キムチ',
        'カロリーハーフマヨネーズ': 'マヨネーズ',
        '八丁味噌': '味噌',
        'みかん。': 'みかん',
        '夏みかん': 'みかん',
        'ミニ・トマト': 'ミニトマト',
        'みそ': '味噌',
        'めんつゆ': '麺つゆ',
        '白モツ': 'モツ',
        'たまご': '卵',
        '焼肉のタレ': '焼肉のたれ',
        '焼き肉のタレ': '焼肉のたれ',
        '焼き肉のたれ': '焼肉のたれ',
        'リンゴ': 'りんご',
        'レモンの輪切り': 'レモン',
        'れもん': 'レモン',
        '檸檬': 'レモン',
        'れんこん': 'レンコン',
        '蓮根': 'レンコン',
        'ローリエ': '月桂樹',
        'ワサビ': 'わさび',
        'わさびチューブ': 'わさび',
        'チューブわさび': 'わさび',
        '山葵': 'わさび',
        '山わさび': 'わさび',
        '練りわさび': 'わさび',
        'わさび漬け': 'わさび',
        '大和芋': '山芋',
    }

## 材料の前処理

In [0]:
# いらない文字を消す
import csv
import glob
import re
import pickle
from tqdm import tqdm

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from PIL import Image
import tensorflow.keras
from keras.preprocessing.text import Tokenizer

from functools import partial
np.load = partial(np.load, allow_pickle=True)  # monkey patch

def clean_text(text):
    #replaced_text = '\n'.join(s.strip() for s in text.splitlines()[2:] if s != '')  # skip header by [2:]
    replaced_text = text.lower()
    replaced_text = re.sub(r'(<.*?>|＜.*?＞|\(.*?\)|（.*?）|\[.*?\]|【.*?】|［.*?］|≪.*?≫|《.*?》|『.*?』|「.*?」)', '', replaced_text) # カッコに囲まれた部分の除去
    replaced_text = re.sub(r'[a-zA-Zａｂｅｘ]', '', replaced_text)       # アルファベットの除去
    replaced_text = re.sub(r'[・：、,\.\&＆/＋\+]', ' ', replaced_text) # 半角スペースに置き換える記号
    replaced_text = re.sub(r'[！\!\"#＃\-=＝%％★☆✩✰○〇◯⚫︎◎●◇◆△▲▼▽※✳＊\*✲■□・・。、,↑↓→←:\.♡⭐⚪♪…\&＆\~～㎝]', '', replaced_text) # 記号除去
    replaced_text = re.sub(r'[\d０-９]', '', replaced_text)        # 数字
    replaced_text = re.sub(r'ー+', 'ー', replaced_text)       # ーーをーに統一
    replaced_text = re.sub(r'[①②③④⑤⑥⑦⑧⑨]', '', replaced_text)       # の除去
    replaced_text = re.sub(r'[＜＞<>]', '', replaced_text)       # の除去
    replaced_text = re.sub(r'[【】]', '', replaced_text)       # 【】の除去
    replaced_text = re.sub(r'[（）()]', '', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', '', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去
    replaced_text = re.sub(r'　', ' ', replaced_text)  # 全角空白の除去
    return replaced_text

In [0]:
import os
import urllib.request
from collections import Counter

from gensim import corpora

def create_dictionary(texts):
    dictionary = corpora.Dictionary(texts)
    return dictionary


def remove_stopwords(docs, stopwords):
    """ストップワード除去"""
    cleaned_docs = []
    for words in docs:
        words = [word for word in words.split(',') if word not in stopwords]
        cleaned_docs.append(','.join(words))
    return cleaned_docs


def most_common(docs, n=100):
    fdist = Counter()
    for doc in docs:
        for word in doc:
            fdist[word] += 1
    common_words = {word for word, freq in fdist.most_common(n)}
    print('{}/{}'.format(n, len(fdist)))
    return common_words

def stop_stopword_from_text(path):
    """text 定義ストップワード取得"""
    with open(path, 'r', encoding='shift-jis') as f:
        stopwords = [line.strip() for line in f if line.strip()]
    return stopwords

def get_stopwords(docs): 
    path = '/content/drive/My Drive/Colab Notebooks/data/stopwords.txt'
    stopwords = stop_stopword_from_text(path)
    stopwords.append(get_rare_words(docs, min_freq=1))
    return stopwords

def get_rare_words(docs, min_freq=1):
    """レアワード除去
    docs : 単語をスペースで区切った文書のリスト ex:[['わたし, は, 元気, です'], [あい, らぶ, ゆー']]
    min_freq: レアな単語とみなす単語の出現回数
    """
    fdist = Counter()
    for words in docs:
        for word in words.split(','):
            fdist[word] += 1
    #common_words = {word for word, freq in fdist.most_common(n)}
    rare_words = [word for word, freq in fdist.items() if freq <= min_freq]
    print('rare word len/{}'.format(len(rare_words)))
    return rare_words

In [0]:
# 材料のテキスト前処理
# [TODO]
# ユーザー辞書に単語追加 分割結果見ながら
# ストップワード除去

def save_text_tokenizer(tokenizer, file_name):
  # saving
  with open(file_name+".pickle", 'wb') as handle:
      pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_text_tokenizer(file_name):
  # loading
  with open(file_name+".pickle", 'rb') as handle:
      return pickle.load(handle)

def normalize(word):
    """単語の正規化"""    
    return NORMALIZE_DICT.get(word, word)

def preprocess_ingredient_tsv(path):
    #mecab = MeCab.Tagger ("-Owakati")
    mecab = mecab_tagger()
    with open(path, 'r') as f:
        texts = []
        for line in f:
            # ゴミの除去
            #print(line.strip())
            line = clean_text(line.strip())
            #print('line:', line)

            # 分かち書き
            results = mecab.parse(line).split('\n')
            #print('results:', results)
            text = []
            for result in results:
                feature = result.split('\t')
                #print(feature)
                if len(feature) != 2: #'EOS'と''を省く
                    continue
                info = feature[1].split(',')
                hinshi = info[0]
                if hinshi not in ('名詞',):
                    continue
                elif re.match(r'[a-zA-Z]+', info[6]):
                    # 英語になってしまう単語はカタカナに変換
                    text.append(info[7])
                elif info[6] == '*':
                    continue
                else:
                    text.append(info[6])
                #print(info[6])
            #print('text', text)
            #texts.append(text)
            text = ','.join(text)
            texts.append(text)

        # stopword
        stopwords = get_stopwords(texts)
        texts = remove_stopwords(texts, stopwords)
            
        # 文書吸う
        #print(tokenizer.document_count)

        # 単語分布確認
        #print(tokenizer.word_counts)

        # 正規化する単語決め、正規化
        print('---------- Normalizing --------------')
        for i, text in enumerate(texts):
            #print(list(map(normalize, text.split(','))))
            texts[i] = list(map(normalize, text.split(',')))
            #print(texts[i])

        # 数の少ない単語、多すぎる単語の除去

        # トークナイズ
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(texts)

        # ベクトル化 npy (y_train, y_test)保存
        # [1, 234, 34, ...] 単語ID のベクトルのリスト
        #texts = []
        id_list = []
        for text in texts:
            #print(text.strip())
            #print(list(map(lambda x: tokenizer.word_index[x], text.split())))
            #print(text)
            id_list.append(list(map(lambda x: tokenizer.word_index[x], text)))
        return tokenizer, id_list

In [39]:
train_tokenizer, y_train = preprocess_ingredient_tsv('/content/drive/My Drive/Colab Notebooks/data/ZAIRYOU.tsv')
save_text_tokenizer(train_tokenizer, '/content/drive/My Drive/Colab Notebooks/data/train_tokenizer')
np.save('/content/drive/My Drive/Colab Notebooks/data/y_train.npy', y_train)
test_tokenizer, y_test = preprocess_ingredient_tsv('/content/drive/My Drive/Colab Notebooks/data/test_ZAIRYOU.tsv')
save_text_tokenizer(test_tokenizer, '/content/drive/My Drive/Colab Notebooks/data/test_tokenizer')
np.save('/content/drive/My Drive/Colab Notebooks/data/y_test.npy', y_test)

rare word len/833
---------- Normalizing --------------
rare word len/565
---------- Normalizing --------------


In [40]:
train_tokenizer = load_text_tokenizer('/content/drive/My Drive/Colab Notebooks/data/train_tokenizer')
test_tokenizer = load_text_tokenizer('/content/drive/My Drive/Colab Notebooks/data/test_tokenizer')
y_train = np.load('/content/drive/My Drive/Colab Notebooks/data/y_train.npy')
y_test = np.load('/content/drive/My Drive/Colab Notebooks/data/y_test.npy')
print(y_train.shape, y_test.shape)

(4000,) (720,)


In [0]:
sorted_words = sorted(train_tokenizer.word_counts.items(), key=lambda x: x[1], reverse=True)
with open('train_vocab.txt', 'w') as f:
    lines = [word for word, _ in sorted_words]
    f.write('\n'.join(lines))

sorted_words = sorted(test_tokenizer.word_counts.items(), key=lambda x: x[1], reverse=True)
with open('test_vocab.txt', 'w') as f:
    lines = [word for word, _ in sorted_words]
    f.write('\n'.join(lines))

In [0]:
words, counts = [], []
for word, count in sorted_words:
    words.append(word)
    counts.append(count)

In [0]:
pd.Series(counts, index=words).plot.bar()

## 画像の前処理

In [0]:
def preprocess_image(img_txt, img_dir, img_size):
    # npz 形式に保存
    imgs = np.zeros((1,224,224,3))
    with open(img_txt, 'r') as f:
        for i, line in tqdm(enumerate(f)):
            img_name = line.strip()
            if not img_name:
                continue

            # リサイズして読み込み
            img = np.asarray(Image.open(img_dir+'/'+img_name).resize(img_size)).reshape(1, 224,224,3)
            if i==0:
                imgs[0] = img
            else:
                imgs = np.vstack([imgs, img])
    print(imgs.shape)
    return imgs
imgs = preprocess_image('/content/drive/My Drive/Colab Notebooks/data/IMG_PATH.tsv', 'train_img', (224,224))
#np.save('/content/drive/My Drive/Colab Notebooks/data/train_img.npy', imgs)
np.savez_compressed('train_img.npz', imgs)
imgs = preprocess_image('/content/drive/My Drive/Colab Notebooks/data/test_IMG_PATH.tsv', 'test_img', (224,224))
np.save('/content/drive/My Drive/Colab Notebooks/data/test_img.npy', imgs)

In [0]:
# img確認
train_imgs = np.load('/content/drive/My Drive/Colab Notebooks/data/train_img.npz')['arr_0']
train_imgs.shape